In [1]:
import stdpopsim
import numpy as np
import pandas as pd
import os
import gzip
import shutil

/Users/jkang/miniconda3/envs/drift-sim/lib/python3.12/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:158: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  time=int(extended_GF.time.head(1) - 1), rate=0
/Users/jkang/miniconda3/envs/drift-sim/lib/python3.12/site-packages/stdpopsim/catalog/HomSap/demographic_models.py:161: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  time=int(extended_GF.time.tail(1) + 1), rate=0


In [2]:
species = stdpopsim.get_species("HomSap")
demographic_model = species.get_demographic_model("OutOfAfrica_4J17")
print([pop.name for pop in demographic_model.populations])

samples = {"YRI": 100, "CEU": 100, "CHB": 100, "JPT": 100} # number of individuals (=number of chromosomes/2)
engine = stdpopsim.get_engine("msprime")

['YRI', 'CEU', 'CHB', 'JPT']


In [5]:
chrno = 1
contig = species.get_contig(f"chr{chrno}", mutation_rate=demographic_model.mutation_rate)
print("mean recombination rate:", f"{contig.recombination_map.mean_rate:.3}")
print("mean mutation rate:", contig.mutation_rate)


mean recombination rate: 1.15e-08
mean mutation rate: 1.44e-08


In [6]:
ts = engine.simulate(demographic_model, contig, samples, seed=0) # took 18 minutes
print(ts.num_sites)

2792638


In [7]:
print(ts.num_samples)
for k, pop in enumerate(ts.populations()):
    print(
        f"The tree sequence has {len(ts.samples(k))} chromosome samples from "
        f"population {k}, which is {pop.metadata['id']}."
)

800
The tree sequence has 200 chromosome samples from population 0, which is YRI.
The tree sequence has 200 chromosome samples from population 1, which is CEU.
The tree sequence has 200 chromosome samples from population 2, which is CHB.
The tree sequence has 200 chromosome samples from population 3, which is JPT.


In [8]:
with open("../data/sim/OutOfAfrica_4J17_" + f"chr{chrno}" + ".vcf", "w") as vcf_file:
    ts.write_vcf(vcf_file)

In [9]:
# following 1kgp paper data handling, we filtered sites to be biallelic, non-singleton, and at least 2kb apart; and MAF >= 0.01
# export as bed
os.system("plink2 --vcf ../data/sim/OutOfAfrica_4J17_" + f"chr{chrno}" +".vcf \
          --bp-space 2000 --min-alleles 2 --max-alleles 2 --maf 0.01 --make-bed \
          --out ../data/sim/OutOfAfrica_4J17_" + f"chr{chrno}"+"_filtered")

PLINK v2.00a4 64-bit (9 Jan 2023)              www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../data/sim/OutOfAfrica_4J17_chr1_filtered.log.
Options in effect:
  --bp-space 2000
  --maf 0.01
  --make-bed
  --max-alleles 2
  --min-alleles 2
  --out ../data/sim/OutOfAfrica_4J17_chr1_filtered
  --vcf ../data/sim/OutOfAfrica_4J17_chr1.vcf

Start time: Sun Apr 14 21:26:45 2024
16384 MiB RAM detected; reserving 8192 MiB for main workspace.
Using up to 8 compute threads.
--vcf: 2792638 variants scanned.
--vcf: ../data/sim/OutOfAfrica_4J17_chr1_filtered-temporary.pgen +
../data/sim/OutOfAfrica_4J17_chr1_filtered-temporary.pvar.zst +
../data/sim/OutOfAfrica_4J17_chr1_filtered-temporary.psam written.
400 samples (0 females, 0 males, 400 ambiguous; 400 founders) loaded from
../data/sim/OutOfAfrica_4J17_chr1_filtered-temporary.psam.
2781968 out of 2792638 variants loaded from
../data/sim/OutOfAfrica_4J17_chr1_filtered-temp

0

In [10]:
# run admixture (set seed to 0)
os.system("admixture -s 0 ../data/sim/OutOfAfrica_4J17_" + f"chr{chrno}" + "_filtered.bed 4 -j6")

****                   ADMIXTURE Version 1.3.0                  ****
****                    Copyright 2008-2015                     ****
****           David Alexander, Suyash Shringarpure,            ****
****                John  Novembre, Ken Lange                   ****
****                                                            ****
****                 Please cite our paper!                     ****
****   Information at www.genetics.ucla.edu/software/admixture  ****

Parallel execution requested.  Will use 6 threads.
Random seed: 0
Point estimation method: Block relaxation algorithm
Convergence acceleration algorithm: QuasiNewton, 3 secant conditions
Point estimation will terminate when objective function delta < 0.0001
Estimation of standard errors disabled; will compute point estimates only.
Size of G: 400x108963
Performing five EM steps to prime main algorithm
1 (EM) 	Elapsed: 25.878	Loglikelihood: -2.73584e+07	(delta): 4.01322e+07
2 (EM) 	Elapsed: 24.658	Loglikelihood: 

0